# Working with FITS files in Python and DS9

**FITS (Flexible Image Transport System)** is the standard data format for storing and sharing astronomical data. It was developed by NASA in the late 1970s to ensure that data collected by different telescopes and instruments could be easily analyzed by astronomers around the world.

Each FITS file contains not only astronomical data (such as pixel values in an image) but also **metadata**, in the form of human-readable “headers.” These headers store critical information about how and when the data were collected (e.g., which telescope, what time, what instrument settings), allowing astronomers to quickly interpret and process the data. Because of this portability and consistency, FITS is a cornerstone of most astronomy research workflows today.

Due to their specialized structure, FITS files cannot be opened with standard photo viewers. Instead, many astronomers use a tool called SAOImageDS9 (often called simply DS9). DS9 provides many of the features astronomers need to explore and analyze FITS files, including custom brightness/contrast scales, region definitions, and coordinate system overlays. Its interactive interface allows researchers to easily examine faint objects, compare multiple images, and carry out basic measurements before moving to more advanced computational analysis in Python.

This notebook will introduce you to working with the FITS file format in Python and DS9. By the end of this demonstration, you should be able to:
- Understand the basic structure of a FITS file.
- Open a FITS file and inspect its header and data.
- Create a simple plot of the image data.

If you're curious to learn more about FITS files, check out the documentation [here](https://fits.gsfc.nasa.gov/fits_documentation.html)!

---

## Reading FITS files with `astropy`

`astropy` provides the `fits` module, which makes it easy to work with FITS files in Python. (The documentation can be found [here](https://docs.astropy.org/en/latest/io/fits/index.html#).) We'll start by importing the module:

In [ ]:
from astropy.io import fits

We'll also import every astronomer's favorite packages, `numpy` and `matplotlib`, to help us work with the data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Opening the file

When you open a FITS file using `fits.open(...)`, Astropy loads the file into a list-like object called an **HDUList**. Each **Header/Data Unit (HDU)** can contain different types of data.

The first element of the HDUList (`hdulist[0]`) always holds the **primary HDU**, which often contains the main data (if it is an image). Subsequent HDUs might contain additional extensions, such as extra images, tables, or other data.

Below, we'll open an example FITS file and use the `.info()` method to view the structure of the HDUList.

In [ ]:
#Path to our file (assuming that the m42/ directory is in the same place as the notebook)
filename = 'm42/M42_g.fits'

#Opening the file
#fits.open() will return an HDUList object
hdulist = fits.open(filename)

#Viewing the structure of the HDUList
hdulist.info()

The `.info()` method shows:
- The index of each HDU (e.g., `[0]`, `[1]`, ...)
- The name of the HDU (if it has one)
- The type of data in the HDU (e.g., `IMAGE`, `BINTABLE`, etc)
- The dimensions and format of the data

We can see that this file only has one HDU, which contains an image with 2048x2048 pixels. If we instead open the `stack.fits` example file, we'll see multiple HDUs:

In [ ]:
stack_hdulist = fits.open('m42/stack.fits')
stack_hdulist.info()

### Inspecting the header

FITS files contain **metadata** in the form of keyword-value pairs stored in **headers**. This metadata can include information about the telescope, instrument, observation date, exposure time, coordinate system, and more.

Each HDU in a FITS file has its own header. To access the header, first index the HDUList to get the HDU you want, then use `.header`. (You can use the special Jupyter Notebook function `display` to make it show up in a nice format.)

In [ ]:
#Accessing the header
header = hdulist[0].header

#Displaying it nicely
display(header)

If you just want to see specific pieces of information, you can print out individual keywords. For example:

In [ ]:
print(header['OBJECT'])      #The object name
print(header['DATE-OBS'])    #Date of observation

If the keyword does not exist, Astropy will raise a `KeyError`. You can check available keywords by reading through the printed header above.

### Extracting and inspecting the data

To access the **data** portion of an HDU, use `.data`. For image HDUs, the data typically consists of a 2D `numpy` array.

Here, we'll extract the image data from the **primary HDU**. If you had additional extensions with data, you could use `hdulist[1].data` or `hdulist[2].data` (etc), depending on which extension holds the relevant image.

In [ ]:
#Accessing the data
data = hdulist[0].data

#You can now work with the data like you would any other 2D array
print("Data type:", data.dtype)
print("Shape of the data array:", data.shape)

Using `matplotlib.imshow()`, we can visualize the data array. Note that we usually reverse the vertical axis by specifying `origin='lower'` to conform to common astronomical conventions. We can also add a **colorbar** to see the range of pixel values in the image.

In [ ]:
plt.imshow(data, origin='lower', cmap='gray')
plt.colorbar(label='pixel value')
plt.show()

It might look like there isn't much to see in this image, but that's not the case. We just need to tweak the colorbar scaling from the default values (which `imshow` determines internally based on the range of pixel values present in the image) to be able to see fainter details. For example, if we decrease the maximum value of the colorbar to 5000 (instead of >60000) and set the minimum value to 0: 

In [ ]:
plt.imshow(data, origin='lower', cmap='gray', vmin=0, vmax=5000)
plt.colorbar(label='pixel value')
plt.show()

Note that this type of scaling adjustment is much easier to do in a specialized FITS viewer like **DS9**. You can even use the Python package [`astromy_ds9`](https://pypi.org/project/astromy-ds9/) to convert DS9 scaling to a `matplotlib.colors.Normalize` object, which can be fed into `imshow`.

### Closing the FITS file

Once you’re done working with the FITS file, it’s good practice to close it to free up system resources, especially if you're working with multiple files.

You can do this manually, or you can use a `with` statement when you first open the file (`with fits.open() as hdulist:`) to automatically close the file after the block of code completes.

In [ ]:
hdulist.close()

---

## Viewing FITS files with DS9

While Python should be your go-to for data analysis, SAOImageDS9 offers an interactive visual interface that simplifies initial data exploration. Instead of writing code for every small adjustment or measurement, you can quickly pan, zoom, adjust brightness/contrast, and define regions within DS9. This hands-on approach often speeds up discovery and troubleshooting before returning to Python for more advanced analysis.

I will demonstrate the main functionalities of DS9 in class. For more details, refer to the [DS9 website](https://sites.google.com/cfa.harvard.edu/saoimageds9/documentation).

### In-class activity

Use DS9 to explore the the provided example data of M103 (a small star cluster in the Cassiopeia constellation) and measure the brightnesses for a small sample of stars in multiple filters. Put your brightnesses in the dictionary below, making sure that the stars are in the same order in each list. 

In [ ]:
m103_data = {
    'g':['REPLACE WITH YOUR STARS'],
    'r':['REPLACE WITH YOUR STARS'],
    'i':['REPLACE WITH YOUR STARS']
}

You will also need to enter the radius of the aperture that you used to measure the star brightnesses (which should be the same for every star in every filter). This will be used to implement a rough **background subtraction** for the brightnesses you measured.

In [ ]:
aperture_radius = 1 #REPLACE WITH YOUR APERTURE RADIUS IN ARCSEC

Finally, run the cells below to perform the background subtraction and make a few plots visualizing your data. Feel free to play around with this code and even add your own plots!

### Calibrations

In astronomy, even the "empty" parts of an image contain some background light — whether from the sky glow, instrument noise, or other sources. To measure a star’s brightness accurately, we have to subtract this background. The cells below perform a rough background subtraction by using measurements made on an empty portion of the M103 images.

In [ ]:
#Measurements of the background that I made from an empty area in each image
background_circle_area = 503.644 #arcsec^2
backgrounds = {'g':266034, 'r':449701, 'i':317585}
backgrounds_per_arcsec2 = {filt:backgrounds[filt]/background_circle_area for filt in backgrounds}

#Central wavelengths for our filters
filter_centers = {'g':477.0, 'r':623.1, 'i':762.5} #nm

In [ ]:
#Performing background subtraction
aperture_area = np.pi*aperture_radius**2
for filt in m103_data:
    bkg = backgrounds_per_arcsec2[filt]*aperture_area
    m103_data[filt] = np.array(m103_data[filt]) - bkg

### Spectral energy distribution (SED) plots

The **spectral energy distribution** of an object shows how much energy that object is emitting at different wavelengths. In our case, we’re only measuring flux in three broad filters (g, r, i), so we have a very limited sampling of each star’s emission rather than a smooth, continuous spectrum. Still, these three points can reveal which filter the star is brightest in, giving us a rough idea of its temperature. 

In [ ]:
#Showing each star on its own plot
for star in range(len(m103_data['g'])):
    SED = [m103_data[filt][star] for filt in m103_data]
    plt.figure(figsize=(6,4))
    plt.plot(filter_centers.values(), SED, color='black', zorder=100, linewidth=3)
    for filt, color in zip(filter_centers, ['tab:green', 'tab:red', 'darkred']):
        plt.axvline(filter_centers[filt], color=color, label=filt, linestyle='--')
    plt.title(f'Star {star}')
    plt.legend()
    plt.show()

In [ ]:
#Showing all stars on the same plot
plt.figure(figsize=(8,6))
for star in range(len(m103_data['g'])):
    SED = [m103_data[filt][star] for filt in m103_data]
    plt.plot(filter_centers.values(), SED, zorder=100, linewidth=3)
for filt, color in zip(filter_centers, ['tab:green', 'tab:red', 'darkred']):
    plt.axvline(filter_centers[filt], color=color, label=filt, linestyle='--')
plt.title(f'All stars')
plt.legend()
plt.show()

#Uncomment this line to save the plot to the same location as the notebook
#plt.savefig('all_SEDs.png', bbox_inches='tight', facecolor='w')

### Flux ratio plot

Flux ratios serve as a proxy for **color**: a star that’s much brighter in g than in r (a large g/r ratio) is effectively *bluer*, often indicating a hotter star, whereas a star with a larger r/i ratio appears *redder* (and possibly cooler). Even though we only have three filters and a handful of stars, plotting flux ratios can highlight which stars stand out as unusually blue or red and reveal any grouping or trend in our sample.

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(np.array(m103_data['g'])/np.array(m103_data['r']),
            np.array(m103_data['r'])/np.array(m103_data['i']))
plt.xlabel('g/r')
plt.ylabel('r/i')
plt.show()

#Uncomment this line to save the plot to the same location as the notebook
#plt.savefig('flux_ratios.png', bbox_inches='tight', facecolor='w')